In [1]:
import sys

sys.path.append("../")
sys.path.append("../MachineLearning/")

import os
from Simulator import Simulator, Multi_simulator
from ForceField.Forcefield import (
    TIP5P_force_field,
    TIP3P_force_field,
    GB_Neck2_force_field,
    Vacuum_force_field,
    Vacuum_force_field_plus_custom,
    Vacuum_force_field,
    OpenFF_forcefield_vacuum_plus_custom,
    OpenFF_forcefield_vacuum,
)
from openmmtorch import TorchForce
from openmm import LangevinMiddleIntegrator
from MachineLearning.GNN_Models import *
from MachineLearning.GNN_Trainer import Trainer
from openmmtorch import TorchForce
from openmm.unit import kelvin, picosecond, picoseconds, bar
import pandas as pd
from helper_functions import (
    create_gnn_sim,
    smiles_to_mol,
    calculate_DGv3,
    set_positions_for_simulation,
    run_minimisation,
)


In [2]:
num_rep = 2
solvent_model_provided = [0,1,]
solvent_dielc_provided = [78.5,4.81]

In [3]:
solvent_model = []
solvent_dielectric = []
for repetition in range(num_rep):
    solvent_model += solvent_model_provided
    solvent_dielectric += solvent_dielc_provided

In [4]:
class GNN3_multisolvent_run_multiple_e(GNN3_Multisolvent_embedding_run_multiple):

    def __init__(
        self,
        fraction=0.5,
        radius=0.4,
        max_num_neighbors=32,
        parameters=None,
        device=None,
        jittable=False,
        num_reps=1,
        gbneck_radius=10.0,
        unique_radii=None,
        hidden=64,
        num_solvents=42,
        hidden_token=128
    ):
        super().__init__(
            fraction,
            radius,
            max_num_neighbors,
            parameters,
            device,
            jittable,
            num_reps,
            gbneck_radius,
            unique_radii,
            hidden,
            78.5,
            num_solvents=num_solvents,
            hidden_token=hidden_token,
        )

    def set_num_reps(
        self,
        num_reps=len(solvent_model),
        solvent_models=solvent_model,
        solvent_dielectric=solvent_dielectric,
    ):
        return super().set_num_reps(num_reps, solvent_models, solvent_dielectric)


class GNN3_multisolvent_e(GNN3_Multisolvent_embedding):

    def __init__(
        self,
        fraction=0.5,
        radius=0.4,
        max_num_neighbors=32,
        parameters=None,
        device=None,
        jittable=False,
        gbneck_radius=10.0,
        unique_radii=None,
        hidden=64,
        num_solvents=42,
        hidden_token=128,
    ):
        super().__init__(
            fraction=fraction,
            radius=radius,
            max_num_neighbors=max_num_neighbors,
            parameters=parameters,
            device=device,
            jittable=jittable,
            unique_radii=unique_radii,
            hidden=hidden,
            num_solvents=num_solvents,
            hidden_token=hidden_token,
        )


setup_dict_multisolv = {
    "trained_model": '../MachineLearning/trained_models/AdamWmodel.model',
    "model": GNN3_multisolvent_e,
    "run_model": GNN3_multisolvent_run_multiple_e,
}

In [5]:
run_name = "_multi_"
run_name += "test"
work_dir = "../"  # directory of the repository
n_interval = 100  # Interval for saving frames in steps
ns = 0.01  # Nanoseconds to run the simulation for

smiles = "CCCOCCO"
mol = smiles_to_mol(smiles)
dg_traj, dgmol = calculate_DGv3(
    mol, len(solvent_model), return_mol=True, pruneRmsThresh=0
)
num_confs = dg_traj.n_frames

In [6]:
gnn_sim = create_gnn_sim(
    smiles,
    cache="/tmp/tmp.cache",
    num_confs=num_confs,
    setup_dict=setup_dict_multisolv,
    additional_parameters={"solvent_model": -1, "solvent_dielectric": 78.5},
    workdir=work_dir,
    run_name=run_name,
)

/localhome/kpaul/.conda/envs/newest_pyg_ot14/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:972: UserWarning: 'GBNeck_interaction.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "
/localhome/kpaul/.conda/envs/newest_pyg_ot14/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:972: UserWarning: 'GBNeck_energies.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "
/localhome/kpaul/.conda/envs/newest_pyg_ot14/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:972: UserWarning: 'GBNeck_energies_no_dielectric.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "
/localhome/kpaul/.conda/envs/newest_pyg_ot14/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:972: UserWa

run_models/CCCOCCO_in_v_r_10_multi_1_gr_10.0_run.pt
Platform:  GPU  ready
Platform:  GPU  ready
Platform:  GPU  ready
Platform:  GPU  ready
All parallel systems have the same forces as the reference System


In [7]:
gnn_sim = set_positions_for_simulation(gnn_sim, dgmol, num_confs=num_confs, iteration=0)
gnn_sim = run_minimisation(gnn_sim)
n_steps = ns / 0.002 * 1000
gnn_sim.run_simulation(n_steps, n_interval, minimize=False)
gnn_sim.save_states(0)

[W manager.cpp:335] Warning: FALLBACK path has been taken inside: runCudaFusionGroup. This is an indication that codegen Failed for some reason.
To debug try disable codegen fallback path via setting the env variable `export PYTORCH_NVFUSER_DISABLE=fallback`
 (function runCudaFusionGroup)
[W graph_fuser.cpp:104] Warning: operator() profile_node %551 : int[] = prim::profile_ivalue(%dims.24)
 does not have profile information (function operator())
